# Bird Co-occurences
### Analyzing Indian ebird data of co-occurences for habitats and geography

We analyze the e-bird data for _co-occurences_ of species, i.e., species occuring together more than expected by their separate frequencies. We can formulate this as the following question

### Question: What causes bird species to occur together?

We shall see that such _co-occurences_ occur for two reasons - geography and habitat.

With more data, one can reverse this analysis and divide India into geographical regions and habitats, from a _bird's eye view_, based on co-occurences. Clearly the present e-bird data is inadequate for this as we only see some regions and habitats in it.

**Code:** This is a jupyter notebook written in scala.

## Setup and loading data

We first add all our dependencies, both our own code and other stuff on which it depends. The code here is  on the repository for some of our other code at [Proving-Ground](https://github.com/siddhartha-gadgil/ProvingGround)

In [1]:
import ammonite.ops._

import ammonite.ops._

The e-bird data has been pre-processed (not in this notebook) and saved in various files. 
We first read how many checklists contain a given bird, and how many contain both birds in a pair.

In [2]:
val data = pwd / 'data

data: pwd.ThisType = root/'home/'gadgil/'code/"e-bird"/'data

In [3]:
val freqF = data / "frequencies.tsv"
val freqs = read.lines(freqF) map (_.split("\t")) map {case Array(a, n) => (a, n.toInt)} sortBy((an) => - an._2)

freqF: data.ThisType = root/'home/'gadgil/'code/"e-bird"/'data/"frequencies.tsv"
freqs: collection.mutable.WrappedArray[(String, Int)] = WrappedArray(
  ("Corvus splendens", 12538),
  ("Acridotheres tristis", 12509),
  ("Halcyon smyrnensis", 11176),
  ("Corvus macrorhynchos", 10495),
  ("Dicrurus macrocercus", 9785),
  ("Ardeola grayii", 9660),
  ("Pycnonotus cafer", 9551),
  ("Pycnonotus jocosus", 8892),
  ("Streptopelia chinensis", 8817),
  ("Orthotomus sutorius", 8780),
  ("Copsychus saularis", 8545),
  ("Centropus sinensis", 8394),
  ("Psilopogon viridis", 8126),
  ("Psittacula krameri", 7858),
  ("Columba livia", 7658),
  ("Milvus migrans", 7596),
  ("Leptocoma zeylonica", 7447),
  ("Microcarbo niger", 7419),
  ("Eudynamys scolopaceus", 7410),
  ("Bubulcus ibis", 7334),
  ("Vanellus indicus", 7301),
  ("Haliastur indus", 6722),
  ("Dendrocitta vagabunda", 6716),
  ("Merops orientalis", 6490),
  ("Egretta garzetta", 5873),
  ("Prinia socialis", 5819),
  ("Cinnyris asiaticus", 5505)

In [4]:
val coF = data /"co-occurences.tsv"
def pairs = read.lines(coF) map (_.split("\t")) map {case Array(a, b, n) => (a, b, n.toInt)}

coF: data.ThisType = root/'home/'gadgil/'code/"e-bird"/'data/"co-occurences.tsv"
defined function pairs

We use scientific names for analysis, but we also have extracted common names since they are nicer to see.

In [5]:
val commonF = data / "common-names.tsv"
val commonNames = read.lines(commonF) map (_.split("\t")) map {case Array(sc, comm) => (sc, comm)} toMap

commonF: data.ThisType = root/'home/'gadgil/'code/"e-bird"/'data/"common-names.tsv"
commonNames: Map[String, String] = Map(
  "Clamator coromandus" -> "Chestnut-winged Cuckoo",
  "Alauda arvensis/gulgula" -> "Eurasian/Oriental Skylark",
  "Ardenna carneipes" -> "Flesh-footed Shearwater",
  "Ichthyophaga ichthyaetus" -> "Gray-headed Fish-Eagle",
  "Otus lettia" -> "Collared Scops-Owl",
  "Aythya nyroca" -> "Ferruginous Duck",
  "Aegithalos iouschistos" -> "Black-browed Tit",
  "Anatidae sp." -> "waterfowl sp.",
  "Cinclidium leucurum" -> "White-tailed Robin",
  "Xenus cinereus" -> "Terek Sandpiper",
  "Horornis flavolivaceus" -> "Aberrant Bush-Warbler",
  "Catreus wallichii" -> "Cheer Pheasant",
  "Accipiter butleri" -> "Nicobar Sparrowhawk",
  "Merops apiaster" -> "European Bee-eater",
  "Psittacula eupatria" -> "Alexandrine Parakeet",
  "Hydroprogne caspia" -> "Caspian Tern",
  "Elachura formosa" -> "Spotted Elachura",
  "Corvus macrorhynchos" -> "Large-billed Crow",
  "Dryocopus jave

Let us look at the 250 most common birds, where how common is measured by how many checklists contain a bird.

In [6]:
val top250 = freqs.take(250).map(_._1)
(top250 map (commonNames)).zipWithIndex

top250: collection.mutable.WrappedArray[String] = WrappedArray(
  "Corvus splendens",
  "Acridotheres tristis",
  "Halcyon smyrnensis",
  "Corvus macrorhynchos",
  "Dicrurus macrocercus",
  "Ardeola grayii",
  "Pycnonotus cafer",
  "Pycnonotus jocosus",
  "Streptopelia chinensis",
  "Orthotomus sutorius",
  "Copsychus saularis",
  "Centropus sinensis",
  "Psilopogon viridis",
  "Psittacula krameri",
  "Columba livia",
  "Milvus migrans",
  "Leptocoma zeylonica",
  "Microcarbo niger",
  "Eudynamys scolopaceus",
  "Bubulcus ibis",
  "Vanellus indicus",
  "Haliastur indus",
  "Dendrocitta vagabunda",
  "Merops orientalis",
  "Egretta garzetta",
  "Prinia socialis",
  "Cinnyris asiaticus",
  "Turdoides striata",
  "Dinopium benghalense",
  "Acridotheres fuscus",
  "Hirundo rustica",
  "Psilopogon haemacephalus",
  "Dicaeum erythrorhynchos",
  "Pavo cristatus",
  "Mesophoyx intermedia",
  "Amaurornis phoenicurus",
  "Turdoides affinis",
  "Aegithina tiphia",
...
res5_1: collection.mutable.W

We shall analyse which birds are seen together, but focussing attention on pairs with both among the top 250. This is because when we map birds to vectors, if we include all birds then the common ones cluster together.

In [7]:
val topSet = top250.toSet
val scientificNames = (commonNames map {case (s, c) => (c, s)}).toMap
val bothSeen = {for {(a, b, n) <- pairs if topSet.contains(a) && topSet.contains(b)} yield((a, b), n)}.toMap

topSet: Set[String] = Set(
  "Psittacula eupatria",
  "Corvus macrorhynchos",
  "Ardea alba",
  "Mycteria leucocephala",
  "Zosterops palpebrosus",
  "Dendrocitta leucogastra",
  "Ploceus philippinus",
  "Artamus fuscus",
  "Cyornis tickelliae",
  "Platalea leucorodia",
  "Dicaeum agile",
  "Picus chlorolophus",
  "Eumyias thalassinus",
  "Dendrocopos nanus",
  "Acridotheres ginginianus",
  "Circus aeruginosus",
  "Merops leschenaulti",
  "Phaenicophaeus viridirostris",
  "Ficedula parva",
  "Ceryle rudis",
  "Streptopelia orientalis",
  "Lonchura malacca",
  "Ocyceros griseus",
  "Circaetus gallicus",
  "Gracula indica",
  "Pycnonotus leucogenys",
  "Aegithina tiphia",
  "Phylloscopus trochiloides",
  "Lanius vittatus",
  "Centropus sinensis",
  "Motacilla cinerea",
  "Ocyceros birostris",
  "Galloperdix spadicea",
  "Pelargopsis capensis",
  "Metopidius indicus",
  "Pericrocotus flammeus",
  "Vanellus indicus",
  "Dicaeum erythrorhynchos",
...
scientificNames: Map[String, String] = M

In [8]:
val p = freqs.toMap

p: Map[String, Int] = Map(
  "Clamator coromandus" -> 42,
  "Alauda arvensis/gulgula" -> 2,
  "Ardenna carneipes" -> 49,
  "Ichthyophaga ichthyaetus" -> 48,
  "Otus lettia" -> 27,
  "Aythya nyroca" -> 188,
  "Aegithalos iouschistos" -> 14,
  "Anatidae sp." -> 55,
  "Cinclidium leucurum" -> 31,
  "Xenus cinereus" -> 221,
  "Horornis flavolivaceus" -> 37,
  "Catreus wallichii" -> 11,
  "Accipiter butleri" -> 1,
  "Merops apiaster" -> 17,
  "Psittacula eupatria" -> 527,
  "Hydroprogne caspia" -> 190,
  "Elachura formosa" -> 20,
  "Corvus macrorhynchos" -> 10495,
  "Dryocopus javensis" -> 174,
  "Aviceda leuphotes" -> 67,
  "Psilopogon malabaricus/haemacephalus" -> 12,
  "Anthus trivialis" -> 250,
  "Ardea alba" -> 3151,
  "Acridotheres grandis" -> 164,
  "Rhyticeros undulatus" -> 32,
  "Tarsiger chrysaeus" -> 28,
  "Dendrocitta formosae" -> 370,
  "Carpodacus sp." -> 10,
  "Rynchops albicollis" -> 19,
  "Locustella certhiola" -> 9,
  "Dicrurus annectans" -> 21,
  "Nisaetus cirrhatus" -> 4

The _co-occurence_ of two species is the ratio of the probability that they are seen together to what this probability would be if they were independent. We don't actually take the ratio but a constant multiple of the ratio as this makes no difference in the analysis.

In [9]:
def coOccurence(a: String, b: String) = 10000.0 * bothSeen((a, b)) / (p(a) * p(b))

defined function coOccurence

We already can see some interesting patterns from the data. Below we see the species that co-occur the most (first by scientific name, then the top 1000 pairs by common name)

In [10]:
val topPairs = for (a <- top250; b <- top250 if a != b) yield (a, b)


topPairs: collection.mutable.WrappedArray[(String, String)] = WrappedArray(
  ("Corvus splendens", "Acridotheres tristis"),
  ("Corvus splendens", "Halcyon smyrnensis"),
  ("Corvus splendens", "Corvus macrorhynchos"),
  ("Corvus splendens", "Dicrurus macrocercus"),
  ("Corvus splendens", "Ardeola grayii"),
  ("Corvus splendens", "Pycnonotus cafer"),
  ("Corvus splendens", "Pycnonotus jocosus"),
  ("Corvus splendens", "Streptopelia chinensis"),
  ("Corvus splendens", "Orthotomus sutorius"),
  ("Corvus splendens", "Copsychus saularis"),
  ("Corvus splendens", "Centropus sinensis"),
  ("Corvus splendens", "Psilopogon viridis"),
  ("Corvus splendens", "Psittacula krameri"),
  ("Corvus splendens", "Columba livia"),
  ("Corvus splendens", "Milvus migrans"),
  ("Corvus splendens", "Leptocoma zeylonica"),
  ("Corvus splendens", "Microcarbo niger"),
  ("Corvus splendens", "Eudynamys scolopaceus"),
  ("Corvus splendens", "Bubulcus ibis"),
  ("Corvus splendens", "Vanellus indicus"),
  ("Corvus sp

In [11]:
val together = topPairs.sortBy((ab) => - coOccurence(ab._1, ab._2)).filter((ab) => (ab._1 < ab._2))

together: collection.mutable.WrappedArray[(String, String)] = WrappedArray(
  ("Hypsipetes leucocephalus", "Psilopogon virens"),
  ("Charadrius alexandrinus", "Charadrius mongolus"),
  ("Anas crecca", "Anas strepera"),
  ("Anas strepera", "Anser indicus"),
  ("Anser indicus", "Tadorna ferruginea"),
  ("Calidris minuta", "Calidris temminckii"),
  ("Phylloscopus xanthoschistos", "Saxicola ferreus"),
  ("Myophonus caeruleus", "Phylloscopus xanthoschistos"),
  ("Pycnonotus leucogenys", "Saxicola ferreus"),
  ("Phoenicurus ochruros", "Sylvia curruca"),
  ("Phylloscopus xanthoschistos", "Pycnonotus leucogenys"),
  ("Anas clypeata", "Anas strepera"),
  ("Myophonus caeruleus", "Psilopogon virens"),
  ("Hypsipetes leucocephalus", "Phylloscopus xanthoschistos"),
  ("Anas clypeata", "Anas crecca"),
  ("Anas strepera", "Tadorna ferruginea"),
  ("Ficedula parva", "Phylloscopus humei"),
  ("Hypsipetes leucocephalus", "Myophonus caeruleus"),
  ("Myophonus caeruleus", "Saxicola ferreus"),
  ("Calidris

In [12]:
show(together map {case (x, y) => (commonNames(x), commonNames(y))} take (100))

WrappedArray(
  ("Black Bulbul", "Great Barbet"),
  ("Kentish Plover", "Lesser Sand-Plover"),
  ("Green-winged Teal", "Gadwall"),
  ("Gadwall", "Bar-headed Goose"),
  ("Bar-headed Goose", "Ruddy Shelduck"),
  ("Little Stint", "Temminck's Stint"),
  ("Gray-hooded Warbler", "Gray Bushchat"),
  ("Blue Whistling-Thrush", "Gray-hooded Warbler"),
  ("Himalayan Bulbul", "Gray Bushchat"),
  ("Black Redstart", "Lesser Whitethroat"),
  ("Gray-hooded Warbler", "Himalayan Bulbul"),
  ("Northern Shoveler", "Gadwall"),
  ("Blue Whistling-Thrush", "Great Barbet"),
  ("Black Bulbul", "Gray-hooded Warbler"),
  ("Northern Shoveler", "Green-winged Teal"),
  ("Gadwall", "Ruddy Shelduck"),
  ("Red-breasted Flycatcher", "Hume's Warbler"),
  ("Black Bulbul", "Blue Whistling-Thrush"),
  ("Blue Whistling-Thrush", "Gray Bushchat"),
  ("Temminck's Stint", "Common Snipe"),
  ("Gray-hooded Warbler", "Great Barbet"),
  ("Gray-hooded Warbler", "White-throated Fantail"),
  ("Blue Whistling-Thrush", "Himalayan Bulbul"

We see that there are two kinds of pairs above:

* Those with the same geography, particularly those confined mainly to the himalayas or the western ghat.
* Those with similar habitats, most obviously water birds. Even better, we see waders co-occur with other waders and swimmers with other swimmers, and grassland birds occur with others.

## Force-directed layout

We visualize the results using a _force-directed layout_, using the implementation in _https://github.com/rsimon/scala-force-layout_

In [13]:
import $ivy.`at.ait.dme.forcelayout::scala-force-layout:0.4.1-SNAPSHOT`

import $ivy.$                                                          

In [14]:
import at.ait.dme.forcelayout._

import at.ait.dme.forcelayout._

In [15]:
val birdNodes = topSet.toVector map ((s) => Node(s, commonNames(s)))

birdNodes: Vector[Node] = Vector(
  Node(
    "Psittacula eupatria",
    "Alexandrine Parakeet",
    1.0,
    0,
    List(),
    List(),
    NodeState(
      Vector2D(0.46720416628248307, 0.41230469175425166),
      Vector2D(0.0, 0.0),
      Vector2D(0.0, 0.0)
    )
  ),
  Node(
    "Corvus macrorhynchos",
    "Large-billed Crow",
    1.0,
    0,
    List(),
    List(),
    NodeState(
      Vector2D(-0.15811037591688681, 0.26769123614542123),
      Vector2D(0.0, 0.0),
      Vector2D(0.0, 0.0)
    )
  ),
  Node(
    "Ardea alba",
    "Great Egret",
    1.0,
    0,
    List(),
    List(),
    NodeState(
      Vector2D(0.43186370130349117, 0.42447067734473887),
      Vector2D(0.0, 0.0),
      Vector2D(0.0, 0.0)
    )
...

In [16]:
object SvgGraphs {
  val labelCss = """
    <style>
      .labl {
        display: none;
      }
      .labelled:hover + .labl {
        display: inline;
      }
    </style>
    """

  def circ(x: Double,
           y: Double,
           t: String,
           r: Double = 3,
           colour: String = "blue",
           fontSize: Int = 12) = {
    s"""
    <circle cx="$x" cy="$y" r="$r" fill="$colour" class = "labelled"/>
    <text x="${x + r +
      r}" y="$y" font-size="$fontSize" text-anchor="start" class = "labl">$t</text>
    """
  }

  def header(width: Int = 1000, height: Int = 400) = {
    s"""
    $labelCss

    <svg version="1.1"
   baseProfile="full"
   width="$width" height="$height"
   xmlns="http://www.w3.org/2000/svg">

   <rect width="80%" height="100%" fill="lightgrey" />

    """ // margin on the right for text
  }

  def scatterPlot(points: Vector[(Double, Double, String)],
                  width: Int = 1000,
                  height: Int = 400,
                  r: Double = 3,
                  colour: String = "blue",
                  fontSize: Int = 12) = {

    val xs     = points map (_._1)
    val ys     = points map (_._2)
    val xmax   = xs.max
    val xmin   = xs.min
    val ymax   = ys.max
    val ymin   = ys.min
    val xScale = (width * 0.6 - (4 * r)) / (xmax - xmin)
    val yScale = (height * 0.9 - (4 * r)) / (ymax - ymin)
    val circles =
      points map {
        case (x, y, t) =>
          circ((x - xmin) * xScale + r + r,
               height - r - r - ((y - ymin) * yScale),
               t,
               r,
               colour,
               fontSize)
      }
    s"""
      <div>
      ${header(width, height)}
      ${circles.mkString("\n")}
      </div>
      """
  }
}


defined object SvgGraphs

In [17]:
import SvgGraphs._
val birdEdges = for ((x, y) <- topPairs) yield Edge(Node(x, commonNames(x)), Node(y, commonNames(y)), coOccurence(x, y))

import SvgGraphs._

birdEdges: collection.mutable.WrappedArray[Edge] = WrappedArray(
  Edge(
    Node(
      "Corvus splendens",
      "House Crow",
      1.0,
      0,
      List(),
      List(),
      NodeState(
        Vector2D(-0.46186029030470177, -0.11642118604791263),
        Vector2D(0.0, 0.0),
        Vector2D(0.0, 0.0)
      )
    ),
    Node(
      "Acridotheres tristis",
      "Common Myna",
      1.0,
      0,
      List(),
      List(),
      NodeState(
        Vector2D(-0.3426308024603991, 0.03834883675682399),
        Vector2D(0.0, 0.0),
        Vector2D(0.0, 0.0)
      )
    ),
    0.5107823403997104
  ),
  Edge(
    Node(
      "Corvus splendens",
      "House Crow",
      1.0,
      0,
      List(),
      List(),
      NodeState(
...

In [18]:
val birdGraph = new SpringGraph(birdNodes, birdEdges)

birdGraph: SpringGraph = at.ait.dme.forcelayout.SpringGraph@18fc811d

In [19]:
birdGraph.doLayout(maxIterations = 3000)


In [20]:
val birdTriples = birdGraph.nodes.toVector map ((n) => (n.state.pos.x, n.state.pos.y, n.label))

birdTriples: Vector[(Double, Double, String)] = Vector(
  (229.33188409839548, 53.55100146792307, "Alexandrine Parakeet"),
  (-16.26651944063152, -16.11762853523099, "Large-billed Crow"),
  (98.83236131356345, -17.21625807371069, "Great Egret"),
  (325.36296875307295, -32.9309891004183, "Painted Stork"),
  (-93.27336880794569, 63.82832671557065, "Oriental White-eye"),
  (-646.2592527557554, 3.9560150445509943, "White-bellied Treepie"),
  (76.62537388155012, -25.912661581304285, "Baya Weaver"),
  (-114.9119224520887, -59.58009955982886, "Ashy Woodswallow"),
  (-116.97599912332045, -23.132571543062966, "Tickell's Blue-Flycatcher"),
  (389.6187132948666, -4.869021082946961, "Eurasian Spoonbill"),
  (-48.75849237510658, 38.712297357600676, "Thick-billed Flowerpecker"),
  (-253.96940563188662, 112.94072076101104, "Lesser Yellownape"),
  (-154.76456303769476, 93.57422514734563, "Verditer Flycatcher"),
  (-78.29102429947811, 106.70222958121043, "Brown-capped Woodpecker"),
  (613.2512223358352

In [ ]:
val bigBirdPlot = scatterPlot(birdTriples, width = 1200, height = 500, r = 2)

### Co-occurence plots

On hovering over points in the pictures below, we can see what bird they represent. One can wander around the picture to see how birds have clustered.

In [ ]:
import almond.interpreter.api._
val bBhtml = DisplayData.html(bigBirdPlot)
display(bBhtml)

To the right of the picture are himalayan birds, and to the left are those from Malabar. The lower part of the  main cluster is mainly water birds.

As geographical factors dominate the picture, it is worth separating these from others such as habitats. To do this, we take a variant of co-occurence where we take the ratio of the probability of a pair of species occuring together to what the  probability would be if their occurence were independent _conditioned on the geographic distribution_.

In [ ]:
val birdLocFreqF = data / "bird-location-freqs.tsv"
val locFreqF = data / "location-freqs.tsv"

val locationFreqs = read.lines(locFreqF) map(_.split("\t")) map {case Array(l, n) => (l, n.toInt)} toMap

In [ ]:
val birdLocFreqs = read.lines(birdLocFreqF) map(_.split("\t")) map {case Array(b, l, n) => ((b, l), n.toInt)} toMap

In [ ]:
val places = locationFreqs.keys.toVector

In [ ]:
def birdLocNum(bird: String, loc: String) = birdLocFreqs.getOrElse((bird, loc), 0).toDouble

In [ ]:
def pairInLoc(x: String, y: String, loc: String) = birdLocNum(x, loc) * birdLocNum(y, loc) / locationFreqs(loc)

In [ ]:
def seenSameLoc(x: String, y: String) = {places map (pairInLoc(x, y, _))}.sum

In [ ]:
def localCoOccurence(x: String, y: String) = bothSeen(x, y) / seenSameLoc(x, y)

In [ ]:
val byLocalCoOcc = {for{ (x, y) <- topPairs if seenSameLoc(x, y) > 0} yield 
                (x, y, localCoOccurence(x, y))} sortBy ((t) => -t._3)

In [ ]:
show(byLocalCoOcc map {case (x, y, n) => (commonNames(x), commonNames(y), n)} take 1000)

In [ ]:
val birdLocEdges = for ((x, y) <- topPairs) yield Edge(Node(x, commonNames(x)), Node(y, commonNames(y)), localCoOccurence(x, y))

In [ ]:
val birdLocGraph = new SpringGraph(birdNodes, birdLocEdges)

In [ ]:
birdLocGraph.doLayout(maxIterations = 3000)

In [ ]:
val birdLocTriples = birdLocGraph.nodes.toVector map ((n) => (n.state.pos.x, n.state.pos.y, n.label))

In [ ]:
val birdLocPlot = scatterPlot(birdLocTriples, width = 1200, height = 500, r = 2)

In [ ]:
display.html(birdLocPlot)

Once the geography has been separated out, one can see that habitats dominate. On the left end are clearly waterbirds, on the right it appears that we have birds that are in forested areas.

### Concluding remarks:

Given the limitations of the present e-bird data, it is clear that we cannot go much beyond this - indeed only the geographical regions and habitats that have the greatest impact on birds are visible. Nevertheless we speculate on analysis that can be done with more data of the same nature.

* We can look for species A and B such that
    * A and B are close in the force-directed graph, but
    * A and B do not co-occur as much as expected given their proximity.
 
 such species are likely to be occupying the same niche.
 
 * As mentioned in the introduction, one can naturally partition into _geographical regions_, _habitats_ and even _micro-habitats_, with geographical region clustering based on geodata as well as clustering of co-occurences.